In [8]:
%%capture
%pip install -r ./requirements.txt

In [2]:
import numpy as np
print(np.__version__)

1.26.4


In [ ]:
import os
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as MetaNames
from langgraph.graph import StateGraph, START, END, MessagesState
from typing import TypedDict

In [74]:
class StateSchema:
    def __init__(self, input: str = "", agent: str = "", output: str = ""):
        self.input = input
        self.agent = agent
        self.output = output

In [75]:
load_dotenv()
model_id = "ibm/granite-34b-code-instruct" 
credentials = {
    "apikey": os.getenv("WATSONX_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com"
}

model = Model(
    model_id=model_id,
    credentials=credentials,
    project_id=os.getenv("WATSONX_PROJECT_ID")
)

In [76]:
def modelo_orquestador(state: StateSchema):
    prompt = f"""
    You are an AI orchestrator agent, and your job is to determine which agent to forward the input to. You will have two possible agents to pass the input to:
    
    - Translator: This agent will be responsible for translating the text into Spanish if necessary.
    - General: When you don't see any specific agent to handle the task, forward the input to this one.
    
    The way to choose the agent is simple: just repeat the input given to you but with a word in front of the input depending on which agent you should forward it to:
    - "translator" when it goes to the translator agent
    - "general" for the general agent.

    Input:{state.input}\n\nAssistant:
    """

    response = model.message(
        assistant_id="YOUR_ASSISTANT_ID",
        session_id="YOUR_SESSION_ID",
        input={'text': prompt}
    ).get_result()

    generated_text = response["output"]["generic"][0]["text"].strip()

    if "traductor" in generated_text.lower():
        state.agent = "traductor"
    else:
        state.agent = "general"
    
    state.output = generated_text
    return state

In [77]:
def modelo_traductor(state: StateSchema):
    prompt = f"""
    You are an AI translator agent, and your job is to translate into Spanish all the input you receive.
    The first word of each input you receive will be "translator," and you should disregard that word, translating everything in the input starting from that word.

    Input:{state.input}\n\nAssistant:
    """

    response = model.message(
        assistant_id="YOUR_ASSISTANT_ID",
        session_id="YOUR_SESSION_ID",
        input={'text': prompt}
    ).get_result()

    generated_text = response["output"]["generic"][0]["text"].strip()
    
    state.output = generated_text
    return state

In [78]:
def modelo_general(state: StateSchema):
    prompt = f"""
    You are a general AI agent.
    The first word of each input you receive will be "general," and you should disregard that word, providing a response for the input starting from that word.

    Input:{state.input}\n\nAssistant:
    """

    response = model.message(
        assistant_id="YOUR_ASSISTANT_ID",
        session_id="YOUR_SESSION_ID",
        input={'text': prompt}
    ).get_result()

    generated_text = response["output"]["generic"][0]["text"].strip()
    
    state.output = generated_text
    return state

In [79]:
def decidir_transicion(state):
    return {"traductor" if state.agent == "traductor" else "general": state}

In [ ]:
grafo = StateGraph(state_schema=StateSchema)

# Añadir nodos al grafo
grafo.add_node("clasificar_intencion", modelo_orquestador)
grafo.add_node("traductor", modelo_traductor)
grafo.add_node("general", modelo_general)

# Definir las transiciones entre nodos
grafo.add_edge(START, "clasificar_intencion")
grafo.add_conditional_edges("clasificar_intencion", decidir_transicion)
grafo.add_edge("traductor", END)
grafo.add_edge("general", END)

# Compilar el agente
agente = grafo.compile()

# Ejemplo de uso

estado = agente.invoke(StateSchema('Hi, can you translate this sentence into Spanish? Where is the bathroom'))
print(estado.output)

AttributeError: 'Model' object has no attribute 'message'

In [53]:
class OverallState(MessagesState):
    pass


class InputState(TypedDict):
    question: str


class OutputState(TypedDict):
    answer: str


class PrivateState(TypedDict):
    secret: str

load_dotenv()
model_id = "ibm/granite-34b-code-instruct" 
credentials = {
    "apikey": os.getenv("WATSONX_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com"
}

model = Model(
    model_id=model_id,
    credentials=credentials,
    project_id=os.getenv("WATSONX_PROJECT_ID")
)

def modelo_orquestador(state):
    print("Estado en modelo_orquestador:", state)  # Para depuración
    if "question" not in state:
        raise KeyError("La clave 'question' no está en el estado.")

    prompt = f"""
    You are an AI orchestrator agent, and your job is to determine which agent to forward the input to. You will have two possible agents to pass the input to:
    
    - Translator: This agent will be responsible for translating the text into Spanish if necessary.
    - General: When you don't see any specific agent to handle the task, forward the input to this one.
    
    The way to choose the agent is simple: just repeat the input given to you but with a word in front of the input depending on which agent you should forward it to:
    - "translator" when it goes to the translator agent
    - "general" for the general agent.

    Input:{state['question']}\n\nAssistant:
    """

    response = model.message(
        assistant_id="YOUR_ASSISTANT_ID",
        session_id="YOUR_SESSION_ID",
        input={'text': prompt}
    ).get_result()

    generated_text = response["output"]["generic"][0]["text"].strip()

    if "traductor" in generated_text.lower():
        state["agent"] = "traductor"
    else:
        state["agent"] = "general"
    
    state["answer"] = generated_text
    return state

def modelo_traductor(state):
    print("Estado en modelo_traductor:", state)  # Para depuración
    if "question" not in state:
        raise KeyError("La clave 'question' no está en el estado.")

    prompt = f"""
    You are an AI translator agent, and your job is to translate into Spanish all the input you receive.
    The first word of each input you receive will be "translator," and you should disregard that word, translating everything in the input starting from that word.

    Input:{state['question']}\n\nAssistant:
    """

    response = model.message(
        assistant_id="YOUR_ASSISTANT_ID",
        session_id="YOUR_SESSION_ID",
        input={'text': prompt}
    ).get_result()

    generated_text = response["output"]["generic"][0]["text"].strip()
    
    state["answer"] = generated_text
    return state

def modelo_general(state):
    print("Estado en modelo_general:", state)  # Para depuración
    if "question" not in state:
        raise KeyError("La clave 'question' no está en el estado.")

    prompt = f"""
    You are a general AI agent.
    The first word of each input you receive will be "general," and you should disregard that word, providing a response for the input starting from that word.

    Input:{state['question']}\n\nAssistant:
    """

    response = model.message(
        assistant_id="YOUR_ASSISTANT_ID",
        session_id="YOUR_SESSION_ID",
        input={'text': prompt}
    ).get_result()

    generated_text = response["output"]["generic"][0]["text"].strip()
    
    state["answer"] = generated_text
    return state

grafo = StateGraph(OverallState, input=InputState, output=OutputState)

# Añadir nodos al grafo
grafo.add_node(modelo_orquestador)
grafo.add_node(modelo_traductor)
grafo.add_node(modelo_general)

# Definir las transiciones entre nodos
grafo.add_edge(START, modelo_orquestador.__name__)
grafo.add_edge(modelo_orquestador.__name__, modelo_traductor.__name__)
grafo.add_edge(modelo_orquestador.__name__, modelo_general.__name__)
grafo.add_edge(modelo_traductor.__name__, END)
grafo.add_edge(modelo_general.__name__, END)

# Compilar el agente
agente = grafo.compile()

# Ejemplo de uso
estado_inicial = {"question": "Hi, can you translate this sentence into Spanish? Where is the bathroom?"}
estado_final = agente.invoke(estado_inicial)
print(estado_final["output"])


Estado en modelo_orquestador: {'messages': []}


KeyError: "La clave 'question' no está en el estado."